In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import files
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
from datetime import timedelta

In [2]:
df = pd.read_csv("/content/clientes_clusterizados.csv")

## Probabilidade de recompra por perfil (cluster)

In [3]:
# Conversão de datas: transformamos a coluna data_da_compra em datetime para calcular intervalos.
df["data_da_compra"] = pd.to_datetime(df["data_da_compra"], errors="coerce")

In [4]:
# Ordenação: organizamos os registros por cliente e data para facilitar o cálculo da próxima compra.
df = df.sort_values(by=["id_do_cliente", "data_da_compra"])

In [5]:
# proxima_data: data da próxima compra de cada cliente.
# dias_ate_proxima: diferença em dias até a próxima compra.

In [6]:
# Próxima compra do cliente
df["proxima_data"] = df.groupby("id_do_cliente")["data_da_compra"].shift(-1)
df["dias_ate_proxima"] = (df["proxima_data"] - df["data_da_compra"]).dt.days

In [7]:
# Target binário (0/1)
df["proxima_compra"] = (df["dias_ate_proxima"] <= 30).astype(int)

In [8]:
# Probabilidade por cluster: calculamos a taxa média de recompra em 30 dias para cada perfil de cliente.
perfil_prob = (
    df.groupby("cluster_nome")["proxima_compra"]
      .mean()
      .reset_index()
      .rename(columns={"proxima_compra":"prob_30d"})
)

In [9]:
# Criar dict para mapear
mapa_prob = dict(zip(perfil_prob["cluster_nome"], perfil_prob["prob_30d"]))

In [10]:
# Criar coluna no dataset original
df["prob_30d_perfil"] = df["cluster_nome"].map(mapa_prob)
df["prob_30d_perfil_fmt"] = (df["prob_30d_perfil"].fillna(0)*100).round(0).astype(int).astype(str) + "%"

In [11]:
df

,id_da_compra,id_do_cliente,data_da_compra,hora_da_compra,origem_ida,destino_ida,origem_retorno,destino_retorno,viacao_ida,viacao_retorno,...,quantidade_passagens,dia_semana,feriado_fds,cluster_num,cluster_nome,proxima_data,dias_ate_proxima,proxima_compra,prob_30d_perfil,prob_30d_perfil_fmt
0,localizador_1,passageiro_1,2018-12-26,15:33:35,lugar_1,lugar_2,sem retorno,sem retorno,viacao_1,Sem retorno,...,1,2,0,3,Econômico,2023-07-06,1653.0,0,0.174603,17%
14369,localizador_14370,passageiro_1,2023-07-06,06:16:58,lugar_28,lugar_111,sem retorno,sem retorno,viacao_184,Sem retorno,...,1,3,0,3,Econômico,2023-07-29,23.0,1,0.174603,17%
9308,localizador_9309,passageiro_1,2023-07-29,11:53:08,lugar_28,lugar_111,sem retorno,sem retorno,viacao_184,Sem retorno,...,1,5,1,3,Econômico,2023-09-19,52.0,0,0.174603,17%
801440,localizador_801441,passageiro_1,2023-09-19,13:41:40,lugar_111,lugar_234,lugar_234,lugar_111,viacao_19,viacao_19,...,2,1,0,3,Econômico,2024-02-04,138.0,0,0.174603,17%
18540,localizador_18541,passageiro_1,2024-02-04,18:09:33,lugar_1236,lugar_1,sem retorno,sem retorno,viacao_10,Sem retorno,...,1,6,1,3,Econômico,NaT,NaN,0,0.174603,17%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713233,localizador_1713234,passageiro_99996,2022-09-17,11:47:33,lugar_104,lugar_34,sem retorno,sem retorno,viacao_13,Sem retorno,...,2,5,1,2,Fiel à rota,2023-09-30,378.0,0,0.232787,23%
1735184,localizador_1735185,passageiro_99996,2023-09-30,20:07:50,lugar_45,lugar_12,sem retorno,sem retorno,viacao_8,Sem retorno,...,1,5,1,2,Fiel à rota,NaT,NaN,0,0.232787,23%
195588,localizador_195589,passageiro_99997,2021-10-12,15:20:32,lugar_34,lugar_8,sem retorno,sem retorno,viacao_10,Sem retorno,...,1,1,1,2,Fiel à rota,NaT,NaN,0,0.232787,23%
195590,localizador_195591,passageiro_99998,2023-03-19,09:57:45,lugar_256,lugar_109,sem retorno,sem retorno,viacao_51,Sem retorno,...,1,6,1,2,Fiel à rota,NaT,NaN,0,0.232787,23%


In [12]:
# deletar as colunas: quantidade_passagens	dia_semana	feriado_fds proxima_data	dias_ate_proxima prob_30d_perfil
df = df.drop(columns=["quantidade_passagens", "dia_semana", "feriado_fds", "proxima_data", "dias_ate_proxima", "prob_30d_perfil"])

In [13]:
# Garantir colunas em string
df["cluster_nome"] = df["cluster_nome"].astype(str)
df["destino_ida"] = df["destino_ida"].astype(str)

In [14]:
# Contar quantos destinos por cluster
contagem = df.groupby(["cluster_nome", "destino_ida"]).size().reset_index(name="qtd_visitas")

In [15]:
# Total de destinos em cada cluster
contagem["total_cluster"] = contagem.groupby("cluster_nome")["qtd_visitas"].transform("sum")

In [16]:
# Calcular percentual
contagem["percentual_destino_cluster"] = (contagem["qtd_visitas"] / contagem["total_cluster"]) * 100

In [17]:
# Manter só as colunas úteis
contagem = contagem[["cluster_nome", "destino_ida", "percentual_destino_cluster"]]

In [18]:
# Juntar no dataframe
df = df.merge(contagem, on=["cluster_nome", "destino_ida"], how="left")

In [19]:
df

,id_da_compra,id_do_cliente,data_da_compra,hora_da_compra,origem_ida,destino_ida,origem_retorno,destino_retorno,viacao_ida,viacao_retorno,valor_total,cluster_num,cluster_nome,proxima_compra,prob_30d_perfil_fmt,percentual_destino_cluster
0,localizador_1,passageiro_1,2018-12-26,15:33:35,lugar_1,lugar_2,sem retorno,sem retorno,viacao_1,Sem retorno,89.09,3,Econômico,0,17%,0.002987
1,localizador_14370,passageiro_1,2023-07-06,06:16:58,lugar_28,lugar_111,sem retorno,sem retorno,viacao_184,Sem retorno,678.70,3,Econômico,1,17%,2.819931
2,localizador_9309,passageiro_1,2023-07-29,11:53:08,lugar_28,lugar_111,sem retorno,sem retorno,viacao_184,Sem retorno,509.90,3,Econômico,0,17%,2.819931
3,localizador_801441,passageiro_1,2023-09-19,13:41:40,lugar_111,lugar_234,lugar_234,lugar_111,viacao_19,viacao_19,823.16,3,Econômico,0,17%,0.268849
4,localizador_18541,passageiro_1,2024-02-04,18:09:33,lugar_1236,lugar_1,sem retorno,sem retorno,viacao_10,Sem retorno,181.69,3,Econômico,0,17%,11.297646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741339,localizador_1713234,passageiro_99996,2022-09-17,11:47:33,lugar_104,lugar_34,sem retorno,sem retorno,viacao_13,Sem retorno,424.82,2,Fiel à rota,0,23%,1.117214
1741340,localizador_1735185,passageiro_99996,2023-09-30,20:07:50,lugar_45,lugar_12,sem retorno,sem retorno,viacao_8,Sem retorno,42.18,2,Fiel à rota,0,23%,0.311843
1741341,localizador_195589,passageiro_99997,2021-10-12,15:20:32,lugar_34,lugar_8,sem retorno,sem retorno,viacao_10,Sem retorno,92.28,2,Fiel à rota,0,23%,2.910405
1741342,localizador_195591,passageiro_99998,2023-03-19,09:57:45,lugar_256,lugar_109,sem retorno,sem retorno,viacao_51,Sem retorno,120.36,2,Fiel à rota,0,23%,0.377494


In [20]:
# Garantir colunas corretas
df["cluster_nome"] = df["cluster_nome"].astype(str)
df["destino_ida"] = df["destino_ida"].astype(str)

## Previsão dos destinos mais visitados por cluster

In [21]:
# Contamos quantas vezes cada destino_ida foi escolhido dentro de cada cluster.
# Em seguida, calculamos a porcentagem desse destino em relação ao total do cluster.
# Isso nos permite prever quais destinos são mais prováveis para cada perfil.

In [22]:
# Contagem por cluster e destino
contagem = df.groupby(["cluster_nome", "destino_ida"]).size().reset_index(name="qtd_visitas")

In [23]:
# Total por cluster
contagem["total_cluster"] = contagem.groupby("cluster_nome")["qtd_visitas"].transform("sum")

In [24]:
# Percentual (número normal)
contagem["percentual_destino_cluster"] = (contagem["qtd_visitas"] / contagem["total_cluster"]) * 100

In [25]:
# Formatar como string 1 casa decimal + %
contagem["percentual_fmt"] = contagem["percentual_destino_cluster"].map(lambda x: f"{x:.1f}%".replace(".", ","))

In [26]:
# Juntar no df principal
df = df.merge(contagem[["cluster_nome", "destino_ida", "percentual_fmt"]],
              on=["cluster_nome", "destino_ida"], how="left")

In [27]:
# Renomear coluna formatada
df = df.rename(columns={"percentual_fmt": "percentual_destino_cluster%"})

In [28]:
df

,id_da_compra,id_do_cliente,data_da_compra,hora_da_compra,origem_ida,destino_ida,origem_retorno,destino_retorno,viacao_ida,viacao_retorno,valor_total,cluster_num,cluster_nome,proxima_compra,prob_30d_perfil_fmt,percentual_destino_cluster,percentual_destino_cluster%
0,localizador_1,passageiro_1,2018-12-26,15:33:35,lugar_1,lugar_2,sem retorno,sem retorno,viacao_1,Sem retorno,89.09,3,Econômico,0,17%,0.002987,"0,0%"
1,localizador_14370,passageiro_1,2023-07-06,06:16:58,lugar_28,lugar_111,sem retorno,sem retorno,viacao_184,Sem retorno,678.70,3,Econômico,1,17%,2.819931,"2,8%"
2,localizador_9309,passageiro_1,2023-07-29,11:53:08,lugar_28,lugar_111,sem retorno,sem retorno,viacao_184,Sem retorno,509.90,3,Econômico,0,17%,2.819931,"2,8%"
3,localizador_801441,passageiro_1,2023-09-19,13:41:40,lugar_111,lugar_234,lugar_234,lugar_111,viacao_19,viacao_19,823.16,3,Econômico,0,17%,0.268849,"0,3%"
4,localizador_18541,passageiro_1,2024-02-04,18:09:33,lugar_1236,lugar_1,sem retorno,sem retorno,viacao_10,Sem retorno,181.69,3,Econômico,0,17%,11.297646,"11,3%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741339,localizador_1713234,passageiro_99996,2022-09-17,11:47:33,lugar_104,lugar_34,sem retorno,sem retorno,viacao_13,Sem retorno,424.82,2,Fiel à rota,0,23%,1.117214,"1,1%"
1741340,localizador_1735185,passageiro_99996,2023-09-30,20:07:50,lugar_45,lugar_12,sem retorno,sem retorno,viacao_8,Sem retorno,42.18,2,Fiel à rota,0,23%,0.311843,"0,3%"
1741341,localizador_195589,passageiro_99997,2021-10-12,15:20:32,lugar_34,lugar_8,sem retorno,sem retorno,viacao_10,Sem retorno,92.28,2,Fiel à rota,0,23%,2.910405,"2,9%"
1741342,localizador_195591,passageiro_99998,2023-03-19,09:57:45,lugar_256,lugar_109,sem retorno,sem retorno,viacao_51,Sem retorno,120.36,2,Fiel à rota,0,23%,0.377494,"0,4%"


In [29]:
# deletar coluna percentual_destino_cluster
df = df.drop(columns=["percentual_destino_cluster"])

In [31]:
# Uma amostra de 1000 linhas por cada cluster
df = df.groupby("cluster_nome").apply(lambda x: x.sample(n=1000, random_state=42)).reset_index(drop=True)

/tmp/ipython-input-3391036127.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("cluster_nome").apply(lambda x: x.sample(n=1000, random_state=42)).reset_index(drop=True)


In [32]:
df

,id_da_compra,id_do_cliente,data_da_compra,hora_da_compra,origem_ida,destino_ida,origem_retorno,destino_retorno,viacao_ida,viacao_retorno,valor_total,cluster_num,cluster_nome,proxima_compra,prob_30d_perfil_fmt,percentual_destino_cluster%
0,localizador_1274595,passageiro_489342,2019-11-25,14:14:30,lugar_43,lugar_140,sem retorno,sem retorno,viacao_9,Sem retorno,148.78,4,Corporativo,0,21%,"0,2%"
1,localizador_1544394,passageiro_49930,2023-04-12,13:22:24,lugar_6,lugar_65,sem retorno,sem retorno,viacao_51,Sem retorno,283.63,4,Corporativo,1,21%,"0,1%"
2,localizador_1728556,passageiro_131158,2022-08-08,18:47:41,lugar_771,lugar_8,sem retorno,sem retorno,viacao_67,Sem retorno,222.10,4,Corporativo,0,21%,"1,8%"
3,localizador_953724,passageiro_402756,2022-06-22,17:16:31,lugar_13,lugar_818,sem retorno,sem retorno,viacao_27,Sem retorno,326.25,4,Corporativo,0,21%,"0,0%"
4,localizador_642593,passageiro_287414,2022-09-06,14:49:47,lugar_398,lugar_6,sem retorno,sem retorno,viacao_26,Sem retorno,67.83,4,Corporativo,0,21%,"8,3%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,localizador_1489663,passageiro_250354,2022-05-12,19:24:42,lugar_72,lugar_76,sem retorno,sem retorno,viacao_82,Sem retorno,107.38,1,Turista,1,25%,"0,4%"
5996,localizador_907682,passageiro_271845,2023-11-19,22:52:50,lugar_13,lugar_1,sem retorno,sem retorno,viacao_7,Sem retorno,129.81,1,Turista,0,25%,"9,7%"
5997,localizador_352398,passageiro_143262,2022-10-24,12:13:00,lugar_73,lugar_21,sem retorno,sem retorno,viacao_8,Sem retorno,149.85,1,Turista,0,25%,"1,6%"
5998,localizador_1139716,passageiro_317258,2015-07-08,21:24:19,lugar_13,lugar_12,sem retorno,sem retorno,viacao_8,viacao_8,105.79,1,Turista,1,25%,"0,3%"


In [37]:
# Fazer download do arquivo
df.to_csv("Amostra_InsightBus.csv", index=False)
files.download("Amostra_InsightBus.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
# Fazer download do arquivo
df.to_csv("InsightBus.csv", index=False)
files.download("InsightBus.csv")

KeyboardInterrupt: 